This is a retrospective data analysis of our ESPN Fantasy Football league data at the end of Week 17 for the 2022 season. The code can be re-used for any past season using the same ESPN API.

I used the [Pro Football Reference](https://www.pro-football-reference.com/years/2022/games.htm) for NFL schedules and [Christian Wendt's ESPN Fantasy Football API](https://github.com/cwendt94/espn-api) to extract ESPN league-specific data.

For data visualization, I used the [Altair Data Visualization Library](https://altair-viz.github.io/). Altair was chosen for its interactivity, especially for its ability to display hover data and the compatibility of interactive plots with GitHub Pages.

One question this analysis can answer is: what was the return on investment of a given players with fantasy draft or waiver budget dollars (FAAB).

See src directory for source code.

In [1]:
import streamlit as st
import espnff_analysis.nfl_schedule as nf
import espnff_analysis.ff_league_data as ff
import espnff_analysis.calc_best_waiver as cbw
from espnff_analysis.plotting import scatterplot_acquisitions
import espnff_analysis.pipeline as pp



## Get NFL Schedule

In [4]:
year=2019

path = r'/Users/jonathancheng/PycharmProjects/espnff/results'

league_id = 1094090

swid = "{F191FB8C-DB2D-4D24-91FB-8CDB2DED249D}"
espn_s2 ='AECJMQHsUHB0FTXdZkw93uY7GRbX8BPnm93Ye6AwvwrMsrZFGg1Lbmi07SWVov2ioN8zGMFDzZiiDSeQCa7WQHaGivGnMfGWLjmfGwkOeLXb5baD1sltp%2B%2BIfHAtl98TpmHgB16ZpGn6g3Bm5vLEA7yDC6HkbD3LSp0E2rGB7hKziLMvZ7mT6ONJFRe8Xp3ApYWSvxPr9cz0pJiI%2FF0blsZ8hyATDJMEyaQ2O%2FypcsViORr6hqYTmXHPuPKnMBfvYC8LQqi1exGw3vnyg6ptsB2Y'




In [5]:
pp.main_pipeline(league_id, year, espn_s2, swid, path)

Fetching league data from espn_api ...


In [7]:
df_proteam_schedule, season_start_date = pp.get_nfl_schedule_info(year)


In [8]:
df_proteam_names = nf.get_team_abbrev()

for col in df_proteam_names.columns:
    df_proteam_names[col] = df_proteam_names[col].str.strip()

In [ ]:
import pandas as pd 

df = nf.mine_nfl_schedule(year_of_interest=year)
df = df[["Week", "Date", "Winner/tie", "Loser/tie"]]
df = pd.melt(
    df,
    id_vars=["Week", "Date"],
    value_vars=["Winner/tie", "Loser/tie"],
    var_name="Winner_Loser",
    value_name="Pro_team",
)
df = df.dropna(subset="Pro_team", axis=0)


# handle special case of Washington, because the team name has changed over years
df["Pro_team"] = df["Pro_team"].str.replace(
    pat=r"^Washington.*", repl="Washington", regex=True
)

df["Pro_team"] = df["Pro_team"].str.replace(
    pat=r"^.*Raiders", repl = "Raiders", regex=True
    )

df["Pro_team_abbrev"] = df["Pro_team"].map(
    dict(zip(df_proteam_names["Pro Team Name"], df_proteam_names["Abbrev"]))
)
df = df[df["Pro_team"].isin(df_proteam_names["Pro Team Name"].tolist())]

In [ ]:
df['Pro_team'].unique().tolist()

In [ ]:
[i for i in df['Pro_team'].unique().tolist() if 'Oakland' in i]

In [9]:
league, activity_ls, wk_ls = pp.fetch_league_data(league_id, year, espn_s2, swid)
df_acq = pp.get_acquisitions_data(activity_ls)
df_draft, drafted_players = ff.build_df_draft(league)
df_rostered = ff.build_df_rostered(league)
df_FA = ff.build_df_FA(league)
# Generate all player stats dataframe, including all Free Agents
df_player_stats = ff.build_df_player_stats(df_rostered, df_FA)
df_draft_stats = ff.build_df_draft_stats(df_draft, df_player_stats)
df_acq_stats = ff.build_df_acq_stats(df_acq, df_player_stats)
df_acq_final = ff.build_df_acq_final(
    season_start_date, df_draft_stats, df_acq_stats, drafted_players
)
# Get player_box_scores from fantasy season
df_player_box_scores = ff.build_df_player_box_scores(league, wk_ls)


Fetching league data from espn_api ...


In [ ]:
df_stints = ff.build_df_stints(
        df_acq_final,
        df_proteam_schedule,
        df_player_stats,
        drafted_players,
        df_player_box_scores,
    )
df_stints["Total points per stint"] = df_stints.apply(
    lambda x: ff.get_total_pts_per_player(
        x["Player"], x["Stint (wks)"], df_player_box_scores
    ),
    axis=1,
).fillna(0)

df_stints, df_player_ffteam = cbw.get_quantile_and_weeks(
    df_stints, df_player_box_scores
)
df_waiver = cbw.get_waiver_data(df_player_ffteam)

In [ ]:
df_proteam_schedule = nf.get_nfl_schedule(year)
season_start_date = nf.get_season_start_date(df_proteam_schedule)

In [ ]:
df_proteam_names = nf.get_team_abbrev()


In [ ]:
for col in df_proteam_names.columns:
    df_proteam_names[col] = df_proteam_names[col].str.strip()

In [ ]:
url = f"https://www.pro-football-reference.com/years/{year_of_interest}/games.htm"
try:
    return pd.read_html(url)[0]
except Exception as e:
    print(f"Error: {e}")
    return None


In [ ]:

for col in df_proteam_names.columns:
    df_proteam_names[col] = df_proteam_names[col].str.strip()

In [ ]:


weeks = [int(i) for i in df["Week"].unique().tolist() if i.isdigit()]
games_all_season = max(weeks)

df = df[df["Week"].isin([str(i) for i in weeks])]
df = df.astype({"Week": int})
# pivot NFL schedule to wideform
df = df[["Week", "Date", "Pro_team_abbrev"]]
df = df.rename(columns={"Pro_team_abbrev": "ProTeam"})
df = df.pivot(index="Week", columns="ProTeam", values="Date")

df = df.applymap(
    lambda x: pd.to_datetime(x, format="%Y-%m-%d") if not pd.isna(x) else x
).applymap(lambda x: pd.Timestamp(x) if not pd.isna(x) else x)

In [ ]:
nf.mine_nfl_schedule(year)['Winner/tie'].unique()

In [ ]:
nf.mine_nfl_schedule(2021)['Winner/tie'].unique()

In [ ]:
g = df_acq_final.groupby(by=["Player"])
df_ls = []
for key in list(g.groups.keys()):
    df_test_acq = g.get_group(key)
    df_test_acq_bid = ff.get_df_test_acq_bid(df_test_acq)
    
    # df_test_acq = (
    #         ff.pivot_df_acq_oneplayer(df_test_acq)
    #         .merge(df_test_acq_bid, how="left")
    #         .drop(["Action_GroupId"], axis=1)
    #     )

    df_test_acq = df_test_acq.reset_index(drop=True)
    df_test_acq = df_test_acq[
        ["Player", "Team", "Position", "ProTeam", "Total points", "Action Timestamp", "Added_Dropped"]
    ]

    # in corner cases where a player is eligible for multiple positions,
    # then take the max (first) highest Total points,
    # usually the alternative position is 0 as an artifact from espn_api
    df_test_acq_dedup = df_test_acq.groupby(by=['Player', 'Team', 'Action Timestamp']).agg(
        {'Total points': 'first'}).reset_index()
    df_test_acq = df_test_acq_dedup.merge(df_test_acq)
    df_test_acq = df_test_acq.sort_values(by="Action Timestamp", ascending=True)

In [ ]:
df_proteam_schedule.columns

In [ ]:
df_test_acq_dedup

In [ ]:
df_test_acq

In [ ]:
import itertools

df_test_acq = df_test_acq.sort_values(by="Timestamp", ascending=True)
df_test_acq = df_test_acq.reset_index(drop=True)
df_test_acq = df_test_acq[
    ["Player", "Team", "Position", "ProTeam", "Total points", "Action Timestamp", "Added_Dropped"]
]

# in corner cases where a player is eligible for multiple positions, then take the max (first) highest Total points, usually the alternative position is 0 as an artifact from espn_api
df_test_acq_dedup = df_test_acq.groupby(by=['Player','Team','Action Timestamp']).agg({'Total points':'first'}).reset_index()
df_test_acq = df_test_acq_dedup.merge(df_test_acq)

df_test_acq = df_test_acq.sort_values(by='Action Timestamp')
index_ls = list(range(df_test_acq.shape[0]))

df_test_acq["Action_GroupId"] = list(
    itertools.chain(*[2 * [i] for i in list(range(df_test_acq.shape[0]))])
)[: len(index_ls)]

df_test_acq = df_test_acq.drop(['Total points','Position'],axis=1)


In [ ]:
df_test_acq

In [ ]:
df_test_acq=(
    df_test_acq.pivot(
        index=["Player", "Team", "ProTeam", "Action_GroupId"],
        columns="Added_Dropped",
    )
    .reset_index()
    .sort_values(by="Action_GroupId")
)

In [ ]:
df_test_acq.columns = [
    f"{i}|{j}" if j != "" else f"{i}" for i, j in df_test_acq.columns
]

df_test_acq = df_test_acq.rename(
    columns={
        "Action Timestamp|ADDED": "Added",
        "Action Timestamp|DROPPED": "Dropped",
    })

df_test_acq

In [ ]:
df_test_acq_dedup = df_test_acq.groupby(by=['Player','Team','Action Timestamp']).agg({'Total points':'first'}).reset_index()
df_test_acq = df_test_acq_dedup.merge(df_test_acq)

In [ ]:
df_test_acq.sort_values(by='Total points',ascending=False).drop_duplicates(keep='first')

In [ ]:
df_test_acq_bid

In [ ]:
df_test_acq.pivot_table(
        index=["Player", "Team", "ProTeam", "Action_GroupId"],
        values="Total points",
        columns="Added_Dropped",
        aggfunc=max
    ).reset_index()

In [ ]:
df_test_acq = (
    df_test_acq.pivot(
        index=["Player", "Team", "ProTeam", "Action_GroupId"],
        columns="Added_Dropped",
    ))

In [ ]:
import itertools

df_test_acq = df_test_acq.sort_values(by="Timestamp", ascending=True)
df_test_acq = df_test_acq.reset_index(drop=True)
df_test_acq = df_test_acq[
    ["Player", "Team", "ProTeam", "Action Timestamp", "Added_Dropped"]
]
index_ls = list(range(df_test_acq.shape[0]))
df_test_acq["Action_GroupId"] = list(
    itertools.chain(*[2 * [i] for i in list(range(df_test_acq.shape[0]))])
)[: len(index_ls)]

In [ ]:
df_test_acq

In [ ]:
g = df_acq_final.groupby(by=["Player"])
df_ls = []
for key in list(g.groups.keys()):
    df_test_acq = g.get_group(key)
    df_test_acq_bid = ff.get_df_test_acq_bid(df_test_acq)
    df_test_acq = (
        ff.pivot_df_acq_oneplayer(df_test_acq)
        .merge(df_test_acq_bid, how="left")
        .drop(["Action_GroupId"], axis=1)
    )
    df_ls.append(df_test_acq)

df_stints = pd.concat(df_ls, axis=0)

df_stints["Added"] = df_stints["Added"].apply(
    lambda x: pd.Timestamp(x) if not pd.isna(x) else x
)
df_stints["Dropped"] = df_stints["Dropped"].apply(
    lambda x: pd.Timestamp(x) if not pd.isna(x) else x
)

df_stints["Stint (wks)"] = df_stints.apply(
    lambda x: get_stints(
        x["ProTeam"], x["Added"], x["Dropped"], df_proteam_schedule
    ),
    axis=1,
)

df_stints["Position"] = df_stints["Player"].map(
    dict(zip(df_player_stats["Player"], df_player_stats["Position"]))
)

# set index on each stint event
df_stints = (
    df_stints.reset_index()
    .reset_index()
    .rename(columns={"level_0": "Stint_id"})
    .drop("index", axis=1)
)

df_stints.loc[df_stints["Player"].isin(drafted_players), "Drafted"] = True
df_stints["Drafted"] = df_stints["Drafted"].fillna(False)

In [ ]:
df_stints = ff.build_df_stints(
    df_acq_final,
    df_proteam_schedule,
    df_player_stats,
    drafted_players,
    df_player_box_scores,
)


In [ ]:
df_stints["Total points per stint"] = df_stints.apply(
    lambda x: ff.get_total_pts_per_player(
        x["Player"], x["Stint (wks)"], df_player_box_scores
    ),
    axis=1,
).fillna(0)

df_stints, df_player_ffteam = cbw.get_quantile_and_weeks(
    df_stints, df_player_box_scores
)
df_waiver = cbw.get_waiver_data(df_player_ffteam)

In [ ]:
pp.main_pipeline(league_id, year, espn_s2, swid, path)

In [ ]:
df_stints,df_player_box_scores,league = pp.main_pipeline(league_id, year, espn_s2, swid)

In [ ]:
df_player_ffteam = cbw.build_df_player_ffteam(df_stints)

ls=[]
for df in [df_stints, df_player_ffteam]:
    df = cbw.get_num_weeks_of_stint(df)
    df = cbw.get_quantile(df, df_player_box_scores)
    ls.append(df)

[df_stints, df_player_ffteam] = ls

In [ ]:
df_player_ffteam

In [ ]:
import itertools

def merge_lists(series):
    return list(set([item for sublist in series for item in sublist]))


In [ ]:
def build_df_player_ffteam(df_stints):
    df_agg_stints = df_stints.groupby(by=['Player', 'Team']).agg({'Total points per stint':'sum', 'Stint (wks)':list}).reset_index()
    df_agg_stints['Stint (wks)'] = df_agg_stints['Stint (wks)'].apply(lambda x: list(itertools.chain(*x)))
    df_player_ffteam = df_agg_stints.merge(df_stints[['Player','Team','ProTeam','Position','Drafted']].drop_duplicates(), how='left')
    return df_player_ffteam

In [ ]:
for idx, x in df_player_ffteam.iterrows():
    if x['Stint (wks)']:
        df_player_ffteam.loc[idx,'quantile'] = cbw.calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'],df_player_box_scores)
    else:
        df_player_ffteam.loc[idx,'quantile'] = 0
        

In [ ]:
df_player_ffteam['Num weeks'] = df_player_ffteam['Stint (wks)'].apply(lambda x: len(x))

In [ ]:
num_top_waiver = 10

In [ ]:
df_waiver = df_player_ffteam[(~df_player_ffteam['Drafted']) & 
                             (df_player_ffteam['Num weeks']>=8) & 
                             (df_player_ffteam['Position'].isin(['QB','RB','WR','TE']))].\
                            sort_values(by='quantile',ascending=False).reset_index(drop=True)

In [ ]:
df_waiver

In [ ]:
from scipy import stats

def calculate_scoring_quantile_per_stint(stint, position, total_points_oneplayer, df_player_box_scores):
    df_allplayers_stint = df_player_box_scores[
        (df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]
    total_position_stint = df_allplayers_stint.groupby(by=['Player']).agg({'Total points': 'sum'}).reset_index()[
        'Total points']
    quantile = stats.percentileofscore(total_position_stint.values, total_points_oneplayer)
    return quantile

In [ ]:
for idx,row in df_player_ffteam[df_player_ffteam['Player'].str.contains('Pickens')].iterrows():
    pass

In [ ]:
row

In [ ]:
stint = row['Stint (wks)']
position = row['Position']
total_points_oneplayer = row['Total points per stint']

In [ ]:
stint

In [ ]:
df_allplayers_stint = df_player_box_scores[
    (df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]
total_position_stint = df_allplayers_stint.groupby(by=['Player']).agg({'Total points': 'sum'}).reset_index()[
    'Total points']

In [ ]:
g=df_allplayers_stint.groupby(by=['Player'])

In [ ]:
for key in list(g.groups.keys()):
    df=g.get_group(key)

In [ ]:
df_allplayers_stint.groupby(by=['Player']).agg({'Total points': 'sum'}).reset_index().sort_values(by='Total points',ascending=False).iloc[0:32]

In [ ]:
# import pandas as pd
# pd.concat([g.get_group(key) for key in list(g.groups.keys())])

In [ ]:
total_position_stint

In [ ]:
s=stats.boxcox(total_position_stint,0)

In [ ]:
import numpy as np
s[~np.isinf(s)].hist(bins=5)

In [ ]:
stats.zscore(total_position_stint.sort_values(ascending=False).values)

In [ ]:
total_position_stint.sort_values(ascending=False)

In [ ]:
df_stints.head()

In [ ]:
for idx, x in df_stints.iterrows():
    if x['Stint (wks)']:
        df_stints.loc[idx,'quantile'] = cbw.calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'],df_player_box_scores)
    else:
        df_stints.loc[idx,'quantile'] = 0
        

In [ ]:
def get_quantile(df, df_player_box_scores):
    for idx, x in df.iterrows():
        if x['Stint (wks)']:
            df.loc[idx,'quantile'] = cbw.calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'],df_player_box_scores)
        else:
            df.loc[idx,'quantile'] = 0
    return df



In [ ]:
df_stints

In [ ]:
plot_title, chart=scatterplot_acquisitions(df_stints, select_acq_method=[True], select_positions=['QB','RB', 'WR', 'TE'])
chart.display()

In [ ]:
import altair as alt
import pandas as pd

select_acq_method=[True] 
select_positions=['QB','RB', 'WR', 'TE']

if select_acq_method is None:
    select_acq_method = [True]

if select_positions is None:
    select_positions = ['RB', 'WR', 'TE']

g = df_stints.groupby(by=["Drafted", "Position"])

df = pd.concat([g.get_group((acq_by_draft, position))
                for acq_by_draft in select_acq_method
                for position in select_positions], axis=0)

if select_acq_method[0]:
    status = "Draft"
else:
    status = "Waiver"

positions = ', '.join(select_positions)

plot_title = f"Position: {positions} , Acquired by: {status}"
selection = alt.selection_multi(fields=["Team"], bind="legend")

color = alt.condition(
    selection,
    alt.Color(
        "Team:N",
        scale=alt.Scale(scheme="tableau20"),
    ),
    alt.value("lightgray"),
)

chart = (
    alt.Chart(df)
    .mark_circle(size=40)
    .encode(
        alt.X("Bid Amount ($)", axis=alt.Axis(grid=False)),
        alt.Y("quantile", axis=alt.Axis(grid=False)),
        color=color,
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
        tooltip=["Player", "Team", "Bid Amount ($)", "Total points per stint"],
    )
    .add_selection(selection)
    .properties(width=450, height=450, title=plot_title)
    .configure_axis(labelFontSize=18, titleFontSize=18)
    .configure_title(fontSize=20)
    .configure_legend(labelFontSize=14, titleFontSize=14)
)

In [ ]:
chart.display()

In [ ]:
import itertools
list(itertools.chain(*ls))

In [ ]:
# Build a scoring algorithm 
# Input: Position, Stint (wks), aTotal points, num weeks played
# Calculate that player's percentile against the rest of his position for the total points scored in the given weeks
# Get 1D array of the particular position within the timespan

In [ ]:
def get_total_pts_per_player(player, stint, df_player_box_scores):
    if stint:
        g = df_player_box_scores.groupby(by="Player")
        df = g.get_group(player)
        return df[df["Week"].isin(stint)]["Total points"].sum()

In [ ]:
# get quantile per stint



In [ ]:
df_stints[df_stints['Player'].str.contains('Daniel Jones')]

In [ ]:
df_stints.head()

In [ ]:
stint = [13, 14, 15, 16, 17, 18]
position = 'QB'
total_points_oneplayer = 102.9

df_temp=df_player_box_scores[(df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]
total_position_stint = df_temp.groupby(by=['Player']).agg({'Total points':'sum'}).reset_index()['Total points']
quantile = total_position_stint.quantile((total_points_oneplayer-total_position_stint.min())/(total_position_stint.max()-total_position_stint.min()))


In [ ]:
from scipy import stats

def calculate_scoring_quantile_per_stint(stint,position,total_points_oneplayer):
    
    df_allplayers_stint = df_player_box_scores[(df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]
    total_position_stint = df_allplayers_stint.groupby(by=['Player']).agg({'Total points':'sum'}).reset_index()['Total points']
    quantile = stats.percentileofscore(total_position_stint.values, total_points_oneplayer)
    return quantile


In [ ]:
df_stints=df_stints.drop(['quantile'],axis=1)

In [ ]:
for idx, x in df_stints.iterrows():
    if x['Stint (wks)']:
        df_stints.loc[idx,'quantile'] = calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint'])
    else:
        df_stints.loc[idx,'quantile'] = 0
    

In [ ]:
df_stints['Num wks'] = df_stints['Stint (wks)'].apply(lambda x: len(x))

In [ ]:
df_stints1 = df_stints[df_stints['Num wks']>=7]

In [ ]:
df_stints1[~df_stints1['Drafted']].sort_values(by=['quantile'],ascending=False)

In [ ]:
for idx,x in df_stints.iterrows():
    if idx == 1:
        break

In [ ]:
x

In [ ]:
stint = x['Stint (wks)']
position = x['Position']
total_points_oneplayer = x['Total points per stint']
print(total_points_oneplayer)

df_allplayers_stint = df_player_box_scores[(df_player_box_scores['Position'] == position) & (df_player_box_scores['Week'].isin(stint))]


In [ ]:
total_position_stint = df_allplayers_stint.groupby(by=['Player']).agg({'Total points':'sum'}).reset_index()['Total points']


In [ ]:
total_position_stint.max()

In [ ]:
total_position_stint.min()

In [ ]:
quantile = stats.percentileofscore(total_position_stint.values, total_points_oneplayer)


In [ ]:
quantile

In [ ]:
# quantile = total_position_stint.quantile((total_points_oneplayer-total_position_stint.min())/(total_position_stint.max()-total_position_stint.min()))

In [ ]:
quantile

In [ ]:
x

In [ ]:
df_stints[df_stints['Drafted']==False].sort_values(by=['quantile'],ascending=False)

In [ ]:
# df_stints1 = df_stints[df_stints['Player'].str.contains('Daniel Jones')].iloc[-1,:]
# df_stints1.apply(lambda x: calculate_scoring_quantile_per_stint(x['Stint (wks)'],x['Position'],x['Total points per stint']),axis=1)
df_stints1.apply(lambda x: calculate_scoring_quantile_per_stint(x['Stint (wks)'], x['Position'], x['Total points per stint']), axis=1)


In [ ]:
a = df_stints[(df_stints['Player'].str.contains('Daniel Jones'))].iloc[-1,:]['Total points per stint']

In [ ]:
a

In [ ]:
stint

In [ ]:
18 in stint

In [ ]:
import pandas as pd

df_temp = pd.concat([df_temp.groupby(by='Week').get_group(i) for i in stint if i!=18])
df_temp['Total points'].sum()/df_temp.shape[0]

In [ ]:
s=df_temp['Total points']

In [ ]:
df_temp.groupby(by=['Player']).get_group('Daniel Jones')

In [ ]:
df_temp.groupby(by=['Player']).get_group('Tom Brady')['Total points'].sum()

In [ ]:
df_temp['Total points']

In [ ]:
df_temp=df_temp.groupby(by=['Player']).agg(sum).reset_index().sort_values(by='Total points', ascending=False)

In [ ]:
s = df_temp['Total points']


In [ ]:
quantile = s.quantile((a-s.min())/(s.max()-s.min()))


In [ ]:
df_temp

In [ ]:
quantile

In [ ]:
player = 'Daniel Jones'
g = df_player_box_scores.groupby(by="Player")
df = g.get_group(player)


In [ ]:
df

In [ ]:


def scatterplot_acquisitions(df_stints, select_acq_method=None, select_positions=None):
    
    if select_acq_method is None:
        select_acq_method = [True]

    if select_positions is None:
        select_positions=['RB','WR','TE']

    g = df_stints.groupby(by=["Drafted", "Position"])

    df = pd.concat([g.get_group((acq_by_draft,position)) 
           for acq_by_draft in select_acq_method
           for position in select_positions],axis=0)

    if select_acq_method[0]:
        status = "Draft"
    else:
        status = "Waiver"

    positions = ', '.join(select_positions)

    plot_title = f"Position: {positions} , Acquired by: {status}"
    selection = alt.selection_multi(fields=["Team"], bind="legend")

    color = alt.condition(
        selection,
        alt.Color(
            "Team:N",
            scale=alt.Scale(scheme="tableau20"),
        ),
        alt.value("lightgray"),
    )

    chart = (
        alt.Chart(df)
        .mark_circle(size=40)
        .encode(
            alt.X("Bid Amount ($)", axis=alt.Axis(grid=False)),
            alt.Y("Total points per stint", axis=alt.Axis(grid=False)),
            color=color,
            opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
            tooltip=["Player", "Team", "Bid Amount ($)", "Total points per stint"],
        )
        .add_selection(selection)
        .properties(width=450, height=450, title=plot_title)
        .configure_axis(labelFontSize=18, titleFontSize=18)
        .configure_title(fontSize=20)
        .configure_legend(labelFontSize=14, titleFontSize=14)
    )
    return plot_title, chart


## Draft Scatterplot

In [ ]:
plot_title, chart = scatterplot_acquisitions(df_stints, select_acq_method=[True], select_positions=['RB','WR','TE'])
chart.display()

## Waiver Scatterplot

- In waiver spending, Flex Player All Stars really spent a fortune and didn't get great return on Khalil Herbert. 
- Jamaal Williams, Jerick McKinnon, Curtis Samuel and D'Onta Foreman, were big value adds

In [ ]:
plot_title, chart = scatterplot_acquisitions(df_stints, select_acq_method=[False], select_positions=['RB','WR','TE'])
chart.display()

Curtis Samuel was the most valuable waiver wire receiver.

In [ ]:
for acq_by_draft in [True,False]:
    for position in df_stints['Position'].unique():
        plot_title,chart = scatterplot_acquisitions(df_stints, select_acq_method=[acq_by_draft], select_positions=[position])
        chart.display()


In [ ]:
df_player_ffteam['Position'].unique()

In [ ]:
df_player_ffteam[(df_player_ffteam['Num weeks']>7) & (df_player_ffteam['Position'].isin(['QB','RB','WR','TE'])) & (~df_player_ffteam['Drafted'])].sort_values(by='quantile',ascending=False)

In [ ]:
df_player_ffteam[(df_player_ffteam['Num weeks']>7) & (df_player_ffteam['Position'].isin(['QB','RB','WR','TE'])) &(df_player_ffteam['quantile']>=90) ].sort_values(by='quantile',ascending=False)